In [5]:
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate,ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
f = open('api_key.txt')
api_key = f.read()
chat = ChatOpenAI(openai_api_key=api_key)

In [6]:
from langchain.output_parsers import CommaSeparatedListOutputParser

In [7]:
output_parser = CommaSeparatedListOutputParser()

In [8]:
format_instructions = output_parser.get_format_instructions() #Nos devuelve las instrucciones que va a pasar al LLM en función del parseador concreto

In [9]:
print(format_instructions)

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [10]:
#Respuesta imaginaria
respuesta = "coche, árbol, carretera"
output_parser.parse(respuesta)

['coche', 'árbol', 'carretera']

In [11]:
#Creamos la plantilla de usuario (human_template) con la concatenación de la variable "request" (la solicitud) y la variable "format_instructions" con 
#las instrucciones adicionales que le pasaremos al LLM
human_template = '{request}\n{format_instructions}'
human_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [12]:
#Creamos el prompt y le damos formato a las variables
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

chat_prompt.format_prompt(request="dime 5 características de los coches americanos",
                   format_instructions = output_parser.get_format_instructions()) #Las instrucciones son las que proporciona el propio parseador

ChatPromptValue(messages=[HumanMessage(content='dime 5 características de los coches americanos\nYour response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`')])

In [13]:
#Transformamos el objeto prompt a una lista de mensajes y lo guardamos en "solicitud_completa" que es lo que pasaremos al LLM finalmente
solicitud_completa = chat_prompt.format_prompt(request="dime 5 características de los coches americanos",
                   format_instructions = output_parser.get_format_instructions()).to_messages()

In [14]:
result = chat.invoke(solicitud_completa)

In [15]:
result.content

'robustos, potentes, grandes, consumen mucha gasolina, diseño llamativo'

In [16]:
# Convertir a la salida esperada
output_parser.parse(result.content)

['robustos',
 'potentes',
 'grandes',
 'consumen mucha gasolina',
 'diseño llamativo']

In [18]:
from langchain.output_parsers import DatetimeOutputParser

In [19]:
output_parser = DatetimeOutputParser()

In [20]:
print(output_parser.get_format_instructions())

Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 0759-02-08T16:06:38.227360Z, 0981-02-17T18:04:01.808796Z, 1871-04-27T16:12:32.936081Z

Return ONLY this string, no other words!


In [22]:
template_text = "{request}\n{format_instructions}"
human_prompt=HumanMessagePromptTemplate.from_template(template_text)

In [23]:
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

In [24]:
print(chat_prompt.format(request="¿Cuándo es el día de la declaración de independencia de los EEUU?",
                   format_instructions=output_parser.get_format_instructions()
                   ))

Human: ¿Cuándo es el día de la declaración de independencia de los EEUU?
Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 1947-05-16T20:11:45.060722Z, 1282-05-09T12:33:19.176849Z, 1951-09-09T20:47:50.346001Z

Return ONLY this string, no other words!


In [25]:
solicitud_completa = chat_prompt.format_prompt(request="¿Cuándo es el día de la declaración de independencia de los EEUU?",
                   format_instructions=output_parser.get_format_instructions()
                   ).to_messages()

In [26]:
result = chat.invoke(solicitud_completa)

In [27]:
result.content

'1776-07-04T00:00:00.000000Z'

In [28]:
output_parser.parse(result.content)

datetime.datetime(1776, 7, 4, 0, 0)

In [29]:
from langchain.output_parsers import OutputFixingParser

output_parser_dates = DatetimeOutputParser()

misformatted = result.content

In [30]:
misformatted

'1776-07-04T00:00:00.000000Z'

In [32]:
new_parser = OutputFixingParser.from_llm(parser=output_parser_dates, llm=chat)

In [33]:
new_parser.parse(misformatted)

datetime.datetime(1776, 7, 4, 0, 0)

In [34]:
system_prompt = SystemMessagePromptTemplate.from_template("Tienes que responder únicamente con un patrón de fechas")
template_text = "{request}\n{format_instructions}"
human_prompt=HumanMessagePromptTemplate.from_template(template_text)

In [35]:
chat_prompt = ChatPromptTemplate.from_messages([system_prompt,human_prompt])

In [36]:
print(chat_prompt.format(request="¿Cuándo es el día de la declaración de independencia de los EEUU?",
                   format_instructions=output_parser_dates.get_format_instructions()
                   ))

System: Tienes que responder únicamente con un patrón de fechas
Human: ¿Cuándo es el día de la declaración de independencia de los EEUU?
Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 0691-11-17T08:38:35.748653Z, 0812-11-01T11:49:04.489517Z, 1532-06-10T09:53:39.340919Z

Return ONLY this string, no other words!


In [37]:
solicitud_completa = chat_prompt.format_prompt(request="¿Cuándo es el día de la declaración de independencia de los EEUU?",
                   format_instructions=output_parser_dates.get_format_instructions()
                   ).to_messages()

In [38]:
result = chat.invoke(solicitud_completa)

In [39]:
result.content

'1776-07-04T00:00:00.000000Z'

In [40]:
output_parser_dates.parse(result.content)

datetime.datetime(1776, 7, 4, 0, 0)